In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from pyspark.sql import SparkSession 

In [3]:
spark = SparkSession.builder.master("local[*]").appName("testing testing 123").getOrCreate()

df = spark.read.options(header=True, inferSchema='True',delimiter=',').csv("data/bank-additional-full.csv")

df.head()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/13 13:00:29 WARN Utils: Your hostname, Yadvendras-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 172.16.0.247 instead (on interface en0)
26/01/13 13:00:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/13 13:00:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Row(age=56, job='housemaid', marital='married', education='basic.4y', default='no', housing='no', loan='no', contact='telephone', month='may', day_of_week='mon', duration=261, campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp.var.rate=1.1, cons.price.idx=93.994, cons.conf.idx=-36.4, euribor3m=4.857, nr.employed=5191.0, y='no')

In [4]:
df.dtypes

[('age', 'int'),
 ('job', 'string'),
 ('marital', 'string'),
 ('education', 'string'),
 ('default', 'string'),
 ('housing', 'string'),
 ('loan', 'string'),
 ('contact', 'string'),
 ('month', 'string'),
 ('day_of_week', 'string'),
 ('duration', 'int'),
 ('campaign', 'int'),
 ('pdays', 'int'),
 ('previous', 'int'),
 ('poutcome', 'string'),
 ('emp.var.rate', 'double'),
 ('cons.price.idx', 'double'),
 ('cons.conf.idx', 'double'),
 ('euribor3m', 'double'),
 ('nr.employed', 'double'),
 ('y', 'string')]

In [5]:
df.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 37| serv

In [6]:
df = df.toDF(*[c.replace(".", "_") for c in df.columns])
df.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 37| serv

In [7]:
print(df.columns)

['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y']


In [8]:
for col, _ctype in df.dtypes:
    if _ctype in ["string"]:
        print("*"*10)
        print(col)
        category_rows = df.select(col).distinct().collect()
        categories = [row[0] for row in category_rows]
        print(categories)

**********
job
['management', 'retired', 'unknown', 'self-employed', 'student', 'blue-collar', 'entrepreneur', 'admin.', 'technician', 'services', 'housemaid', 'unemployed']
**********
marital
['unknown', 'divorced', 'married', 'single']
**********
education
['high.school', 'unknown', 'basic.6y', 'professional.course', 'university.degree', 'illiterate', 'basic.4y', 'basic.9y']
**********
default
['unknown', 'no', 'yes']
**********
housing
['unknown', 'no', 'yes']
**********
loan
['unknown', 'no', 'yes']
**********
contact
['cellular', 'telephone']
**********
month
['jun', 'aug', 'may', 'mar', 'oct', 'jul', 'nov', 'apr', 'dec', 'sep']
**********
day_of_week
['fri', 'thu', 'tue', 'wed', 'mon']
**********
poutcome
['success', 'failure', 'nonexistent']
**********
y
['no', 'yes']


In [9]:
category_rows = df.select("job").distinct().collect()
categories = [row[0] for row in category_rows]
print(categories)

['management', 'retired', 'unknown', 'self-employed', 'student', 'blue-collar', 'entrepreneur', 'admin.', 'technician', 'services', 'housemaid', 'unemployed']


In [10]:
# from backend.core.discovery import Problem

In [11]:
import pyspark.sql.functions as sf

null_counts = df.select([sf.count(sf.when(sf.col(c).isNull(), c)).alias(c) for c in df.columns])

null_counts.show()
# print([c for c in df.columns])



+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|age|job|marital|education|default|housing|loan|contact|month|day_of_week|duration|campaign|pdays|previous|poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+
|  0|  0|      0|        0|      0|      0|   0|      0|    0|          0|       0|       0|    0|       0|       0|           0|             0|            0|        0|          0|  0|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+--------+-----+--------+--------+------------+--------------+-------------+---------+-----------+---+



In [12]:
null_counts['age']

Column<'age'>

In [13]:
for c in null_counts.columns:
    # print(type(null_counts))
    # print(type(c))
    # print(null_counts.select(str(c)).first()[0])
    if null_counts.select(str(c)).first()[0] > 0:
        print(c)

In [14]:
# cols_with_nulls = [c for c in df.columns if null_counts[c] > 0]
# print(f"Columns with nulls: {cols_with_nulls}")

Auto Feature Generator

In [15]:
from backend.core.features.auto_feature_generator import AutoFeatureGenerator

In [16]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.master("local[*]").appName("testing testing 123").getOrCreate()

In [17]:
df = spark.read.options(header=True, inferSchema='True',delimiter=',').csv("data/bank-additional-full.csv")

df = df.toDF(*[c.replace(".", "_").replace(" ", "_") for c in df.columns])
df.show(5)

df.head()

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 37| serv

Row(age=56, job='housemaid', marital='married', education='basic.4y', default='no', housing='no', loan='no', contact='telephone', month='may', day_of_week='mon', duration=261, campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp_var_rate=1.1, cons_price_idx=93.994, cons_conf_idx=-36.4, euribor3m=4.857, nr_employed=5191.0, y='no')

In [18]:
feature_gen = AutoFeatureGenerator(spark)
    
# Generate all features
df_with_features = feature_gen.generate_all_features(
    df,
    include_numerical=True,
    include_interactions=False,  # Set to True for interactions
    include_binning=True,
    include_datetime=True,
    include_string=True
)

Generating numerical features...
Generating binning features...
Generating datetime features...
Generating string features...
Total features generated: 111


In [19]:
from pyspark.sql import functions as F

In [20]:
df.head(5)

[Row(age=56, job='housemaid', marital='married', education='basic.4y', default='no', housing='no', loan='no', contact='telephone', month='may', day_of_week='mon', duration=261, campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp_var_rate=1.1, cons_price_idx=93.994, cons_conf_idx=-36.4, euribor3m=4.857, nr_employed=5191.0, y='no'),
 Row(age=57, job='services', marital='married', education='high.school', default='unknown', housing='no', loan='no', contact='telephone', month='may', day_of_week='mon', duration=149, campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp_var_rate=1.1, cons_price_idx=93.994, cons_conf_idx=-36.4, euribor3m=4.857, nr_employed=5191.0, y='no'),
 Row(age=37, job='services', marital='married', education='high.school', default='no', housing='yes', loan='no', contact='telephone', month='may', day_of_week='mon', duration=226, campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp_var_rate=1.1, cons_price_idx=93.994, cons_conf_idx=-36.4, euri

In [21]:
df.withColumn("y_yes_proc", F.when(F.col("y")=="yes", 1).otherwise(0))

df.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|
| 37| serv

26/01/13 13:40:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 312400 ms exceeds timeout 120000 ms
26/01/13 13:40:04 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/13 13:40:06 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
df_1 = df.withColumn("y_yes_proc", F.when(F.col("y")=="yes", 1).otherwise(0))

In [ ]:
df_1.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+----------+
|age|      job|marital|  education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp_var_rate|cons_price_idx|cons_conf_idx|euribor3m|nr_employed|  y|y_yes_proc|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+---+----------+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0| no|         0|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|     149|       1|  999|       0|nonexistent|         1.1|        93.994|       

26/01/12 20:31:40 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 286815 ms exceeds timeout 120000 ms
26/01/12 20:31:40 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/12 20:31:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [ ]:
# Show results
print("\nOrigianal columns:", df.columns)
print("\nNew columns:", df_with_features.columns)
print("\nGenerated features:", feature_gen.get_generated_features())

df_with_features.show(5, truncate=False)


Origianal columns: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y']

New columns: ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y', 'age_log', 'age_sqrt', 'age_square', 'age_cube', 'duration_log', 'duration_sqrt', 'duration_square', 'duration_cube', 'campaign_log', 'campaign_sqrt', 'campaign_square', 'campaign_cube', 'pdays_log', 'pdays_sqrt', 'pdays_square', 'pdays_cube', 'previous_log', 'previous_sqrt', 'previous_square', 'previous_cube', 'emp_var_rate_log', 'emp_var_rate_sqrt', 'emp_var_rate_square', 'emp_var_rate_cube', 'cons_price_idx_log', 'cons_price_idx_sqrt', 'cons_price_idx_square', 'cons_pr

In [ ]:
from backend.core.metrics.base import Metrics
from backend.core.discovery import Problem
from backend.core.features.process import target_processing

In [ ]:
problem = Problem(target="y", type="classification", desired_result="yes")

In [ ]:
from pyspark.sql import functions as F

In [ ]:
from copy import deepcopy

In [ ]:
df.head(5)

[Row(age=56, job='housemaid', marital='married', education='basic.4y', default='no', housing='no', loan='no', contact='telephone', month='may', day_of_week='mon', duration=261, campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp_var_rate=1.1, cons_price_idx=93.994, cons_conf_idx=-36.4, euribor3m=4.857, nr_employed=5191.0, y='no'),
 Row(age=57, job='services', marital='married', education='high.school', default='unknown', housing='no', loan='no', contact='telephone', month='may', day_of_week='mon', duration=149, campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp_var_rate=1.1, cons_price_idx=93.994, cons_conf_idx=-36.4, euribor3m=4.857, nr_employed=5191.0, y='no'),
 Row(age=37, job='services', marital='married', education='high.school', default='no', housing='yes', loan='no', contact='telephone', month='may', day_of_week='mon', duration=226, campaign=1, pdays=999, previous=0, poutcome='nonexistent', emp_var_rate=1.1, cons_price_idx=93.994, cons_conf_idx=-36.4, euri

In [ ]:
df.select(F.count_distinct("job")).collect()[0][0]

12

In [ ]:
_col = "education"
# df.select(
#     F.count_distinct(_col).alias(_col+"_count_distinct"),
#     F.count(_col).alias(_col+"_count"),
#     F.count(F.when(F.col(_col).isNull(), 1)).alias(_col+"_null_count")
#     ).first().asDict()

grouped_stats = df.groupBy(_col).agg(
    F.count_distinct(_col).alias(_col+"_count_distinct"),
    F.count(_col).alias(_col+"_count"),
    F.count(F.when(F.col(_col).isNull(), 1)).alias(_col+"_null_count")
    ).collect()

[row.asDict() for row in grouped_stats]

NameError: name 'df' is not defined

In [ ]:
from backend.core.discovery import SchemaChecks, ColumnTypes

In [ ]:
getattr(ColumnTypes, "categorical").value

['string', 'char', 'varchar']

In [ ]:
schema_checks = SchemaChecks(df, problem=problem)

schema_checks.categorical_checks()

NameError: name 'df' is not defined

In [ ]:
schema_checks.numerical_checks()

NameError: name 'schema_checks' is not defined

In [ ]:
schema_checks.bool_checks()

NameError: name 'schema_checks' is not defined

In [ ]:
df_new = target_processing(df, problem)
df_new.show(5)

NameError: name 'df' is not defined

In [ ]:
dimensions = df_new.columns
dimensions.remove(problem.target)
print(dimensions)

NameError: name 'df_new' is not defined

In [ ]:
calculator = Metrics(dataframe=df_new, problem=problem)

_ = calculator.calculate(dimensions=['job'])#categorical_features[:-1])
_.show()

In [ ]:
df.columns

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp_var_rate',
 'cons_price_idx',
 'cons_conf_idx',
 'euribor3m',
 'nr_employed',
 'y']

In [ ]:
df[["age"]].dtypes[0][1]

'int'

In [ ]:
isinstance(df["age"].dtype, str)

False

In [ ]:
from pyspark.sql import functions as psf
_ = df.groupby(["y"]).agg((psf.count(psf.col('y'))/df.count()))
_.show()

In [ ]:
categorical_type = ["string"]

categorical_features = [_col for _col, _type in df.dtypes if _type in categorical_type]

categorical_features[:-1]

In [ ]:
df.select("y").distinct().collect()

In [ ]:
df.select("y").count()

NameError: name 'df' is not defined

In [ ]:
categorical_features


NameError: name 'categorical_features' is not defined